In [ ]:
#melakukan import untuk function yang akan digunakan beserta dataset
import numpy as np 
import pandas as pd
import datetime 
from sklearn.metrics import mean_squared_error
from numpy import sqrt

import matplotlib.pyplot as plt 
%matplotlib inline


from statsmodels.tsa.ar_model import AutoReg
from statsmodels.tsa.arima.model import ARIMA
from fbprophet import Prophet

from statsmodels.graphics.tsaplots import plot_pacf, plot_acf
from statsmodels.tsa.stattools import kpss
import statsmodels.api as sm

import warnings
warnings.filterwarnings("ignore")

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
#melakukan import untuk data yang akan digunakan
tsales=pd.read_csv("/kaggle/input/competitive-data-science-predict-future-sales/sales_train.csv")

# Preprocessing Data

In [ ]:
#function untuk melihat informasi mengenai data set train 
def overview(tsales):
    print('SHAPE:\n',tsales.shape)
    print('COLUMN NAMES:\n', tsales.columns.tolist())
    print('UNIQUE VALUES PER COLUMN:\n', tsales.nunique())
    print('COLUMNS WITH MISSING DATA:\n',tsales.isnull().sum())
    print('SAMPLE:\n',tsales.sample(10))
    print('INFO:\n',tsales.info())

In [ ]:
overview(tsales)

In [ ]:
#membuat plot untuk melihat apakah terdapat outlier di item cnt day
plt.figure(figsize=(2,4))
plt.xlim(-100, 2500)
plt.boxplot(x=tsales.item_cnt_day)

In [ ]:
#membuat plot untuk melihat apakah terdapat outlier di item price
plt.figure(figsize=(2,4))
plt.xlim(-100, 2500)
plt.boxplot(x=tsales.item_price)

In [ ]:
#menghilangkan outlier untuk data item cnt day yang lebih dari 900 dan data yang bernilai negatif (outlier)
#menghilangkan outlier untuk data item price yang lebih dari 800
tsales = tsales[tsales.item_cnt_day<900]
tsales = tsales[tsales.item_cnt_day>0]
tsales = tsales[tsales.item_price<100000]
print('sales', tsales.shape)

In [ ]:
#melakukan groupby jumlah pembelian per hari menjadi jumlah pembelian per bulan
total=tsales.groupby(["date_block_num"])["item_cnt_day"].sum()
index = pd.DataFrame(total)

In [ ]:
#melakukan list untuk value dari setiap hasil grouping per bulan
data = list(index.item_cnt_day.values)

#value akan digunakan untuk proses pembandingan
len(data)

# Formatting Variabel

In [ ]:
#memngantikan tipe data date yang ada di dataset dari object menjadi date
tsales["date"] = pd.to_datetime(tsales["date"], format = "%d.%m.%Y")
tsales.set_index('date')

In [ ]:
tsales.info()

# Prophet

In [ ]:
#mengambil data sesuai tanggal start dan end yang sudah ditentukan
total.index=pd.date_range(start = '2013-01-01',end='2015-10-01', freq = 'MS')
total=total.reset_index()

#menentukan x dan y untuk metode prophet beserta konfigurasi
#melakukan modeling, fit, dan juga predict
total.columns=['ds','y']
model = Prophet(yearly_seasonality=True)
model.fit(total)
future = model.make_future_dataframe(periods = 17, freq = 'MS')  
forecast = model.predict(future)

In [ ]:
#memunculkan hasil dari predict dan data asli menjadi plot
prophetList = list(index.item_cnt_day.values)
plt.figure(figsize=(10,6))
plt.plot(prophetList, label='Original')
plt.plot(forecast['yhat'], label="Predicted")
plt.plot(forecast['yhat_lower'], label="yhat_lower")
plt.plot(forecast['yhat_upper'], label="yhat_upper")
plt.title('Prophet')
plt.show()

#melakukan perbandingan antara hasil predict dengan data asli
rmse = sqrt(mean_squared_error(prophetList,forecast['yhat'][0:34]))
print('Prophet RMSE: %.1f' % rmse)

# Auto Regressive

In [ ]:
#melakukan modeling, fit, dan juga predict
model = AutoReg(data, 2)
model_fit = model.fit()
yhat = model_fit.predict(10, len(data)+ 26) 
yhatList = list(yhat)
ARList = list(data)

#memunculkan hasil dari predict dan data asli menjadi plot
plt.figure(figsize=(10,6))
plt.plot(yhatList, label='Predicted')
plt.plot(ARList, label='Original')
plt.title('AR')
plt.show()

#melakukan perbandingan antara hasil predict dengan data asli
rmse = sqrt(mean_squared_error(ARList,yhatList[0:34]))
print('AR RMSE: %.1f' % rmse)

# ARIMA

In [ ]:
#model ARIMA membutuhkan nilai p, d, dan q

#mencari nilai p menggunakan plot partial Autocorrelation
plot_pacf(index['item_cnt_day'])

In [ ]:
#mencari nilai d menggunakan tes KPPS untuk menentukan dataset stationary atau bukan
print ('Results of KPSS Test:')
print('Null Hypothesis: Data is Stationary/Trend Stationary')
print('Test Statistic > Critical Value => Reject Null')
print('P-Value =< Alpha(.05) => Reject Null\n')
kpsstest = kpss(index['item_cnt_day'], regression='c')
kpss_output = pd.Series(kpsstest[0:3], index=['Test Statistic','p-value','Lags Used'])
for key,value in kpsstest[3].items():
    kpss_output[f'Critical Value {key}'] = value
print (kpss_output, '\n')

In [ ]:
#Karena hasil dari tes kpss mengatakan dataset merupakan non-stationary, maka dilakukan proses untuk 
#membuat data menjadi stationary dengan cara differencing dan mendapatkan nilai d
plt.rcParams.update({'figure.figsize':(9,7), 'figure.dpi':120})

fig, axes = plt.subplots(3, 2, sharex=True)
axes[0, 0].plot(index['item_cnt_day']); axes[0, 0].set_title('Original Series')
plot_acf(index['item_cnt_day'], ax=axes[0, 1])

axes[1, 0].plot(index['item_cnt_day'].diff()); axes[1, 0].set_title('1st Order Differencing')
plot_acf(index['item_cnt_day'].diff().dropna(), ax=axes[1, 1])

axes[2, 0].plot(index['item_cnt_day'].diff().diff()); axes[2, 0].set_title('2nd Order Differencing')
plot_acf(index['item_cnt_day'].diff().diff().dropna(), ax=axes[2, 1])

plt.show()
#mencapai stationary setelah 1 kali differencing karena di differencing yang kedua, terdapat lag  
#yang menyampai negatif yang berarti sudah termasuk over-differenced

In [ ]:
#mencari nilai q menggunakan plot Autocorrelation 
plot_acf(index['item_cnt_day'])

In [ ]:
#melakukan modeling, fit, predict menggunakan nilai yang sudah dicari sebelumnya (p,d,q)
model = ARIMA(data, order=(4,1,2))
model_fit = model.fit()
yhat = model_fit.predict(1, len(data)+ 17)
yhatList = list(yhat)
arimaList = list(data)

#memunculkan hasil dari predict dan data asli menjadi plot
plt.figure(figsize=(10,6))
plt.plot(yhatList, label='Predicted')
plt.plot(arimaList, label='Original')
plt.title('ARIMA')
plt.show()

#melakukan perbandingan antara hasil predict dengan data asli
rmse = sqrt(mean_squared_error(arimaList,yhatList[0:34]))
print('ARIMA RMSE: %.1f' % rmse)